In [122]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,294 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpa

In [123]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()
from pyspark import SparkFiles

#read in data from S3 Buckets

url='https://project3group5.s3.amazonaws.com/SVDemo_combined.csv'

spark.sparkContext.addFile(url)

df=spark.read.csv(SparkFiles.get("SVDemo_combined.csv"),sep=",",inferSchema=True)
df.show()

+-------+----+------------------+------+--------------------+-----+------------+------------+------------+
|    _c0| _c1|               _c2|   _c3|                 _c4|  _c5|         _c6|         _c7|         _c8|
+-------+----+------------------+------+--------------------+-----+------------+------------+------------+
|company|year|              race|gender|        job_category|count|valuation   |ownership   |eeo-1 status|
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|         Technicians|    0|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|       Sales workers|    0|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  ma

In [137]:
from pyspark.sql.types import IntegerType, StructField, StructType,DecimalType
from pyspark.sql.functions import col,regexp_replace,when,expr,regexp_extract,column,split,concat_ws,row_number,count
df.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [138]:
#create new column to remove $, m, and b in column _c6, then cast decimaltype, and times 1000000 or 1000000000 according to m or b
df2=df.withColumn("evaluation",split(col("_c6"),"[$]").getItem(1)).withColumn("evaluation2",split(col("evaluation"),"m").getItem(0).cast(DecimalType())*1000000).withColumn("evaluation3",split(col("evaluation"),"b").getItem(0).cast(DecimalType())*1000000000)
df2.show()


+-------+----+------------------+------+--------------------+-----+------------+------------+------------+----------+-----------+-----------+
|    _c0| _c1|               _c2|   _c3|                 _c4|  _c5|         _c6|         _c7|         _c8|evaluation|evaluation2|evaluation3|
+-------+----+------------------+------+--------------------+-----+------------+------------+------------+----------+-----------+-----------+
|company|year|              race|gender|        job_category|count|valuation   |ownership   |eeo-1 status|      null|       null|       null|
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|       $1.1b|     private|         YES|      1.1b|       null| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|       $1.1b|     private|         YES|      1.1b|       null| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|       $1.1b|     private|         YES|      1.1b|       null| 1000000000|
|23and

In [139]:
#fill null with 0
df3=df2.fillna(0)
df3.show()

+-------+----+------------------+------+--------------------+-----+------------+------------+------------+----------+-----------+-----------+
|    _c0| _c1|               _c2|   _c3|                 _c4|  _c5|         _c6|         _c7|         _c8|evaluation|evaluation2|evaluation3|
+-------+----+------------------+------+--------------------+-----+------------+------------+------------+----------+-----------+-----------+
|company|year|              race|gender|        job_category|count|valuation   |ownership   |eeo-1 status|      null|          0|          0|
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|       $1.1b|     private|         YES|      1.1b|          0| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|       $1.1b|     private|         YES|      1.1b|          0| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|       $1.1b|     private|         YES|      1.1b|          0| 1000000000|
|23and

In [140]:
#create new column that add value of evaluation2 and evaluation3
df4=df3.withColumn("evaluations",df3["evaluation2"]+df3["evaluation3"])
df4.show()

+-------+----+------------------+------+--------------------+-----+------------+------------+------------+----------+-----------+-----------+-----------+
|    _c0| _c1|               _c2|   _c3|                 _c4|  _c5|         _c6|         _c7|         _c8|evaluation|evaluation2|evaluation3|evaluations|
+-------+----+------------------+------+--------------------+-----+------------+------------+------------+----------+-----------+-----------+-----------+
|company|year|              race|gender|        job_category|count|valuation   |ownership   |eeo-1 status|      null|          0|          0|          0|
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|       $1.1b|     private|         YES|      1.1b|          0| 1000000000| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|       $1.1b|     private|         YES|      1.1b|          0| 1000000000| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|       $1

In [141]:
#remove the first row
df5=df4.filter(df3._c0!="company")
df5.show()

+-------+----+------------------+------+--------------------+---+-----+-------+---+----------+-----------+-----------+-----------+
|    _c0| _c1|               _c2|   _c3|                 _c4|_c5|  _c6|    _c7|_c8|evaluation|evaluation2|evaluation3|evaluations|
+-------+----+------------------+------+--------------------+---+-----+-------+---+----------+-----------+-----------+-----------+
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|  0|$1.1b|private|YES|      1.1b|          0| 1000000000| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|  1|$1.1b|private|YES|      1.1b|          0| 1000000000| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|  7|$1.1b|private|YES|      1.1b|          0| 1000000000| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|         Technicians|  0|$1.1b|private|YES|      1.1b|          0| 1000000000| 1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Sales workers|  0|$1.1b|private|YES|

In [142]:
#rename all columns
df6=df5.withColumnRenamed("_c0","company").withColumnRenamed("_c1","year").withColumnRenamed("_c2","race").withColumnRenamed("_c3","gender").withColumnRenamed("_c4","job_category").withColumnRenamed("_c5","count").withColumnRenamed("_c6","valuation").withColumnRenamed("_c7","ownership").withColumnRenamed("_c8","eeo_1 status").withColumnRenamed("evaluations","valuations")
df6.show()

+-------+----+------------------+------+--------------------+-----+---------+---------+------------+----------+-----------+-----------+----------+
|company|year|              race|gender|        job_category|count|valuation|ownership|eeo_1 status|evaluation|evaluation2|evaluation3|valuations|
+-------+----+------------------+------+--------------------+-----+---------+---------+------------+----------+-----------+-----------+----------+
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|    $1.1b|  private|         YES|      1.1b|          0| 1000000000|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|    $1.1b|  private|         YES|      1.1b|          0| 1000000000|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|    $1.1b|  private|         YES|      1.1b|          0| 1000000000|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|         Technicians|    0|    $1.1b|  private|         YES|      1.1b|       

In [143]:
#drop columns 
df7=df6.drop("evaluation2").drop("evaluation3").drop("evaluation")
df7.show()

+-------+----+------------------+------+--------------------+-----+---------+---------+------------+----------+
|company|year|              race|gender|        job_category|count|valuation|ownership|eeo_1 status|valuations|
+-------+----+------------------+------+--------------------+-----+---------+---------+------------+----------+
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|    $1.1b|  private|         YES|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|    $1.1b|  private|         YES|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|    $1.1b|  private|         YES|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|         Technicians|    0|    $1.1b|  private|         YES|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|       Sales workers|    0|    $1.1b|  private|         YES|1000000000|
|23andMe|2016|Hispanic_or_Latino|  male|Administrative su...|    0|    $1.1b|  private|         YES|1000

In [144]:
#cast integertype to column count

df8=df7.withColumn("count",df7["count"].cast(IntegerType()))
df8.printSchema()

root
 |-- company: string (nullable = true)
 |-- year: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- job_category: string (nullable = true)
 |-- count: integer (nullable = true)
 |-- valuation: string (nullable = true)
 |-- ownership: string (nullable = true)
 |-- eeo_1 status: string (nullable = true)
 |-- valuations: decimal(22,0) (nullable = true)



In [145]:
#create pandas dataframe
import pandas as pd
pandas_df=df8.toPandas()
pandas_df.head()

,company,year,race,gender,job_category,count,valuation,ownership,eeo_1 status,valuations
0,23andMe,2016,Hispanic_or_Latino,male,Executives,0.0,$1.1b,private,YES,1000000000
1,23andMe,2016,Hispanic_or_Latino,male,Managers,1.0,$1.1b,private,YES,1000000000
2,23andMe,2016,Hispanic_or_Latino,male,Professionals,7.0,$1.1b,private,YES,1000000000
3,23andMe,2016,Hispanic_or_Latino,male,Technicians,0.0,$1.1b,private,YES,1000000000
4,23andMe,2016,Hispanic_or_Latino,male,Sales workers,0.0,$1.1b,private,YES,1000000000


In [146]:
#import drive to load data
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [147]:
#export csv data
pandas_df.to_csv("/content/drive/MyDrive/combined_decimal_integer_csv.csv",index=False,encoding='utf_8_sig')


In [149]:
#group by company and get the sum of count of male and female
df9=df8.groupBy("company","year","ownership","valuations","eeo_1 status","gender").pivot("gender").sum("count").orderBy("company")
df9.show()

+-------------+----+---------+------------+------------+------+------+------+------+
|      company|year|ownership|  valuations|eeo_1 status|gender|  null|female|  male|
+-------------+----+---------+------------+------------+------+------+------+------+
|      23andMe|2016|  private|  1000000000|         YES|  null|   594|  null|  null|
|      23andMe|2016|  private|  1000000000|         YES|female|  null|   298|  null|
|      23andMe|2016|  private|  1000000000|         YES|  male|  null|  null|   296|
|Adobe Systems|2016|   public| 64000000000|         YES|  male|  null|  null| 14244|
|Adobe Systems|2016|   public| 64000000000|         YES|female|  null|  6661|  null|
|Adobe Systems|2016|   public| 64000000000|         YES|  null| 20905|  null|  null|
|       Airbnb|2016|  private| 31000000000|         YES|  null|  5235|  null|  null|
|       Airbnb|2016|  private| 31000000000|         YES|  male|  null|  null|  2943|
|       Airbnb|2016|  private| 31000000000|         YES|female|  

In [150]:
#fill all the null with 0 and group by company
df10=df9.drop("gender").drop("null").fillna(0).groupBy("company","ownership","year","valuations","eeo_1 status").sum("female","male")
df10.show()

+--------------------+---------+----+------------+------------+-----------+---------+
|             company|ownership|year|  valuations|eeo_1 status|sum(female)|sum(male)|
+--------------------+---------+----+------------+------------+-----------+---------+
|       Adobe Systems|   public|2016| 64000000000|         YES|       6661|    14244|
|               Intel|   public|2016|171000000000|         YES|      41259|   120744|
|                Uber|  private|2016| 63000000000|         YES|       3472|     8298|
|              Airbnb|  private|2016| 31000000000|         YES|       2292|     2943|
|                Lyft|  private|2016|  8000000000|         YES|       1431|     1970|
|              PayPal|   public|2016| 52000000000|         YES|      12833|    17102|
|           Pinterest|  private|2016| 12000000000|         YES|       1058|     1417|
|             Twitter|   public|2016| 11000000000|         YES|       3179|     6058|
|               Apple|   public|2016|754000000000|    

In [151]:
#group by company and get the sum of count for each race
df11=df8.groupBy("company","race").pivot("race").sum("count").orderBy("company")
df11.show()

+-------------+--------------------+------------------------------+-----+-------------------------+------------------+-----------------------------------+--------------+-----------------+-----+
|      company|                race|American_Indian_Alaskan_Native|Asian|Black_or_African_American|Hispanic_or_Latino|Native_Hawaiian_or_Pacific_Islander|Overall_totals|Two_or_more_races|White|
+-------------+--------------------+------------------------------+-----+-------------------------+------------------+-----------------------------------+--------------+-----------------+-----+
|      23andMe|   Two_or_more_races|                          null| null|                     null|              null|                               null|          null|               28| null|
|      23andMe|  Hispanic_or_Latino|                          null| null|                     null|                38|                               null|          null|             null| null|
|      23andMe|               

In [152]:
#fill all the null with 0 and group by company
df12=df11.drop("race").fillna(0).groupBy("company").sum("American_Indian_Alaskan_Native","Asian","Black_or_African_American","Hispanic_or_Latino","Native_Hawaiian_or_Pacific_Islander","Overall_totals","Two_or_more_races","White")
df12.show()

+--------------------+-----------------------------------+----------+------------------------------+-----------------------+----------------------------------------+-------------------+----------------------+----------+
|             company|sum(American_Indian_Alaskan_Native)|sum(Asian)|sum(Black_or_African_American)|sum(Hispanic_or_Latino)|sum(Native_Hawaiian_or_Pacific_Islander)|sum(Overall_totals)|sum(Two_or_more_races)|sum(White)|
+--------------------+-----------------------------------+----------+------------------------------+-----------------------+----------------------------------------+-------------------+----------------------+----------+
|              NetApp|                                 67|      5184|                           633|                    774|                                      26|              18901|                   209|     12008|
|                Lyft|                                 18|       657|                           206|                    

In [153]:
#group by company and get the sum of count for each job_category
df13=df8.groupBy("company","job_category").pivot("job_category").sum("count").orderBy("company")
df13.show()

+-------------+--------------------+----------------------+-------------+----------+--------+---------------+-------------+-------------+---------------+-----------+------+--------------------+----------+
|      company|        job_category|Administrative support|Craft workers|Executives|Managers|Previous_totals|Professionals|Sales workers|Service workers|Technicians|Totals|laborers and helpers|operatives|
+-------------+--------------------+----------------------+-------------+----------+--------+---------------+-------------+-------------+---------------+-----------+------+--------------------+----------+
|      23andMe|            Managers|                  null|         null|      null|     130|           null|         null|         null|           null|       null|  null|                null|      null|
|      23andMe|       Craft workers|                  null|            0|      null|    null|           null|         null|         null|           null|       null|  null|        

In [154]:
#fill all the null with 0 and group by company
df14=df13.drop("job_category","Previous_totals","Totals").fillna(0).groupBy("company").sum("Administrative support","Craft workers","Executives","Managers","Professionals","Sales workers","Service workers","Technicians","laborers and helpers","operatives")
df14.show()

+--------------------+---------------------------+------------------+---------------+-------------+------------------+------------------+--------------------+----------------+-------------------------+---------------+
|             company|sum(Administrative support)|sum(Craft workers)|sum(Executives)|sum(Managers)|sum(Professionals)|sum(Sales workers)|sum(Service workers)|sum(Technicians)|sum(laborers and helpers)|sum(operatives)|
+--------------------+---------------------------+------------------+---------------+-------------+------------------+------------------+--------------------+----------------+-------------------------+---------------+
|              NetApp|                        182|                 0|             72|         2006|              7764|              1302|                   0|              36|                        0|              0|
|                Lyft|                        836|                 0|             30|          518|              1366|          

In [157]:
#join df14 and df12
df15=df14.join(df12,on=["company"],how="left_outer")
df15.show()

+--------------------+---------------------------+------------------+---------------+-------------+------------------+------------------+--------------------+----------------+-------------------------+---------------+-----------------------------------+----------+------------------------------+-----------------------+----------------------------------------+-------------------+----------------------+----------+
|             company|sum(Administrative support)|sum(Craft workers)|sum(Executives)|sum(Managers)|sum(Professionals)|sum(Sales workers)|sum(Service workers)|sum(Technicians)|sum(laborers and helpers)|sum(operatives)|sum(American_Indian_Alaskan_Native)|sum(Asian)|sum(Black_or_African_American)|sum(Hispanic_or_Latino)|sum(Native_Hawaiian_or_Pacific_Islander)|sum(Overall_totals)|sum(Two_or_more_races)|sum(White)|
+--------------------+---------------------------+------------------+---------------+-------------+------------------+------------------+--------------------+------------

In [158]:
#join df10 and df15
df16=df10.join(df15,on=["company"],how="left_outer")
df16.show()


+--------------------+---------+----+------------+------------+-----------+---------+---------------------------+------------------+---------------+-------------+------------------+------------------+--------------------+----------------+-------------------------+---------------+-----------------------------------+----------+------------------------------+-----------------------+----------------------------------------+-------------------+----------------------+----------+
|             company|ownership|year|  valuations|eeo_1 status|sum(female)|sum(male)|sum(Administrative support)|sum(Craft workers)|sum(Executives)|sum(Managers)|sum(Professionals)|sum(Sales workers)|sum(Service workers)|sum(Technicians)|sum(laborers and helpers)|sum(operatives)|sum(American_Indian_Alaskan_Native)|sum(Asian)|sum(Black_or_African_American)|sum(Hispanic_or_Latino)|sum(Native_Hawaiian_or_Pacific_Islander)|sum(Overall_totals)|sum(Two_or_more_races)|sum(White)|
+--------------------+---------+----+-------

In [159]:
#export to csv
ml_df=df16.toPandas()

ml_df.to_csv("/content/drive/MyDrive/ml_csv.csv",index=False,encoding='utf_8_sig')
